<a href="https://colab.research.google.com/github/mamekin05108/signatecup2024summer/blob/main/lightcat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install catboost

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import re
import lightgbm as lgb
from sklearn import metrics
from sklearn.metrics import f1_score, roc_auc_score
from sklearn.model_selection import StratifiedKFold

import xgboost as xgb
from xgboost import XGBClassifier


from sklearn.calibration import calibration_curve, CalibratedClassifierCV
from sklearn.metrics import roc_auc_score
from catboost import CatBoostClassifier, Pool
from sklearn.isotonic import IsotonicRegression


# warningsを非表示にする
warnings.filterwarnings("ignore")

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [4]:
df_train = pd.read_csv("/content/drive/MyDrive/Signate/2024summer/fdata/df_train_allnum.csv")
df_test = pd.read_csv("/content/drive/MyDrive/Signate/2024summer/fdata/df_test_allnum.csv")
ss = pd.read_csv("/content/drive/MyDrive/Signate/2024summer/data/sample_submit.csv", header=None)

In [5]:
target = "ProdTaken"


cols_category = [
    "Gender",
    "ProductPitched",
    "Designation",
    "MaritalStatus",
    "OwnCar",
    "Occupation",
    "TypeofContact",
    "Agebin",
    "Incomebin",
    "Occupation_Designation"
]


cols_category=[]

In [6]:
train_y = df_train[target]
train_x = df_train.drop(target, axis=1)

In [7]:
test_x = df_test
test_x = test_x.drop(target, axis=1)

In [8]:
test_x.head()

,id,Age,TypeofContact,DurationOfPitch,Gender,NumberOfPersonVisiting,NumberOfFollowups,NumberOfTrips,Passport,MonthlyIncome,...,PitchSatisfactionScore_5,Designation_AVP,Designation_Executive,Designation_Manager,Designation_Senior Manager,Designation_VP,MaritalStatus_未婚,MaritalStatus_独身,MaritalStatus_結婚,MaritalStatus_離婚
0,3489,0.938701,1,-0.212100,0,-1.908669,0.361786,2.196148,0,1.863454,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,3490,-0.922728,1,-0.356740,1,-1.908669,0.361786,0.459028,1,-0.742793,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
2,3491,-1.439792,1,-0.790661,1,-1.908669,0.361786,-1.278092,0,-1.272115,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,3492,-1.853443,0,-1.079941,0,-1.908669,0.361786,-1.278092,0,-1.273769,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
4,3493,0.214812,0,-1.079941,0,-1.908669,0.361786,-1.278092,0,-1.155267,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


# モデル


In [9]:
!pip install optuna


In [10]:
import optuna

In [11]:
# OptunaでLightGBMのパラメータをチューニングする関数
def objective_lgb(trial, X, y):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 100, 5000),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-4, 1e-1),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.5, 1.0),
        'subsample': trial.suggest_uniform('subsample', 0.5, 1.0),
        'random_seed': 0
    }

    model = lgb.LGBMClassifier(**params)
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=815)
    auc_scores = []

    for train_idx, val_idx in cv.split(X, y):
        trn_x, trn_y = X.iloc[train_idx], y.iloc[train_idx]
        val_x, val_y = X.iloc[val_idx], y.iloc[val_idx]
        model.fit(trn_x, trn_y, eval_set=[(val_x, val_y)], eval_metric='auc', callbacks=[lgb.early_stopping(100, verbose=False)])
        preds = model.predict_proba(val_x)[:, 1]
        auc_scores.append(roc_auc_score(val_y, preds))

    return np.mean(auc_scores)

Best LightGBM parameters: {'n_estimators': 3310, 'learning_rate': 0.054222174375827735, 'colsample_bytree': 0.5021459390774948, 'subsample': 0.8448409944410017}

Best LightGBM parameters: {'n_estimators': 851, 'learning_rate': 0.02289938761734719, 'colsample_bytree': 0.5046482989824799, 'subsample': 0.519032610565576}

In [12]:
# OptunaでCatBoostのパラメータをチューニングする関数
def objective_cat(trial, X, y, cols_category):
    params = {
        'iterations': trial.suggest_int('iterations', 100, 5000),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-4, 1e-1),
        'colsample_bylevel': trial.suggest_uniform('colsample_bylevel', 0.5, 1.0),
        'random_seed': 0,
        'verbose': 100,
        'use_best_model': True
    }

    model = CatBoostClassifier(**params)
    cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=815)
    auc_scores = []

    for train_idx, val_idx in cv.split(X, y):
        trn_x, trn_y = X.iloc[train_idx], y.iloc[train_idx]
        val_x, val_y = X.iloc[val_idx], y.iloc[val_idx]
        train_pool = Pool(data=trn_x, label=trn_y, cat_features=cols_category)
        val_pool = Pool(data=val_x, label=val_y, cat_features=cols_category)
        model.fit(train_pool, eval_set=val_pool, verbose=False)
        preds = model.predict_proba(val_x)[:, 1]
        auc_scores.append(roc_auc_score(val_y, preds))

    return np.mean(auc_scores)

Best CatBoost parameters: {'iterations': 2429, 'learning_rate': 0.015201103669488437, 'colsample_bylevel': 0.6320629073348337}

In [13]:
# Optunaのスタディを作成し、最適なパラメータを見つける
def tune_params(X, y, cols_category):
    def opt_lgb(trial):
        return objective_lgb(trial, X, y)

    def opt_cat(trial):
        return objective_cat(trial, X, y, cols_category)

    study_lgb = optuna.create_study(direction='maximize')
    study_lgb.optimize(opt_lgb, n_trials=50)
    print(f"Best LightGBM parameters: {study_lgb.best_params}")

    study_cat = optuna.create_study(direction='maximize')
    study_cat.optimize(opt_cat, n_trials=50)
    print(f"Best CatBoost parameters: {study_cat.best_params}")

    return study_lgb.best_params, study_cat.best_params

In [14]:
# ベースモデルの学習とメタ特徴量の生成
def train_base_models_with_optuna(X, y, cols_category):
    best_params_lgb, best_params_cat = tune_params(X, y, cols_category)

    print("Final LightGBM Parameters:", best_params_lgb)
    print("Final CatBoost Parameters:", best_params_cat)

    base_models = {
        'LightGBM': lgb.LGBMClassifier(**best_params_lgb),
        'CatBoost': CatBoostClassifier(**best_params_cat, early_stopping_rounds=100)
    }

    meta_features, oof_predictions = train_base_models(X, y, base_models, cols_category)

    return base_models, meta_features, oof_predictions

In [15]:
def train_base_models(X, y, models, cols_category, n_splits=5):
    meta_features = np.zeros((len(X), len(models)))
    oof_predictions = np.zeros(len(X))

    cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=815)

    for i, (model_name, model) in enumerate(models.items()):
        print(f"Training {model_name}...")
        oof_predictions_model = np.zeros(len(X))

        for fold, (trn_idx, val_idx) in enumerate(cv.split(X, y), start=1):
            trn_x, trn_y = X.iloc[trn_idx], y.iloc[trn_idx]
            val_x, val_y = X.iloc[val_idx], y.iloc[val_idx]

            if isinstance(model, lgb.LGBMClassifier):
                model.fit(
                    trn_x, trn_y,
                    eval_set=[(val_x, val_y)],
                    callbacks=[lgb.early_stopping(100, verbose=False)],
                    categorical_feature=cols_category,
                )
            elif isinstance(model, CatBoostClassifier):
                train_pool = Pool(data=trn_x, label=trn_y, cat_features=cols_category)
                val_pool = Pool(data=val_x, label=val_y, cat_features=cols_category)
                model.fit(train_pool, eval_set=val_pool, verbose=False)
            else:
                model.fit(trn_x, trn_y)

            oof_predictions_model[val_idx] = model.predict_proba(val_x)[:, 1]

            auc = roc_auc_score(val_y, oof_predictions_model[val_idx])
            print(f"  Fold {fold}, AUC: {auc:.4f}")

        meta_features[:, i] = oof_predictions_model
        oof_predictions += oof_predictions_model / len(models)

        auc = roc_auc_score(y, oof_predictions_model)
        print(f"{model_name} OOF AUC: {auc:.4f}")

    return meta_features, oof_predictions

In [16]:
def train_meta_model(meta_features, y, meta_model, n_splits=5):
    oof_predictions = np.zeros(len(y))
    cv = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=815)

    for fold, (trn_idx, val_idx) in enumerate(cv.split(meta_features, y), start=1):
        trn_x, trn_y = meta_features[trn_idx], y.iloc[trn_idx]
        val_x, val_y = meta_features[val_idx], y.iloc[val_idx]

        meta_model.fit(trn_x, trn_y)
        oof_predictions[val_idx] = meta_model.predict_proba(val_x)[:, 1]

        auc = roc_auc_score(val_y, oof_predictions[val_idx])
        print(f"Meta Model Fold {fold}, AUC: {auc:.4f}")

    auc = roc_auc_score(y, oof_predictions)
    print(f"Meta Model OOF AUC: {auc:.4f}")

    return oof_predictions

In [29]:
def stacking_predict(X, base_models, meta_model):
    meta_features = np.column_stack([model.predict_proba(X)[:, 1] for _, model in base_models.items()])
    return meta_model.predict_proba(meta_features)[:, 1]

In [18]:
# ベースモデルの学習とメタ特徴量の生成
base_models, meta_features, oof_predictions_base = train_base_models_with_optuna(train_x, train_y, cols_category)


[I 2024-08-18 16:21:10,280] A new study created in memory with name: no-name-a330f39c-008d-405d-a899-ffd6ad7ab723


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 397, number of negative: 2394
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000567 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 683
[LightGBM] [Info] Number of data points in the train set: 2791, number of used features: 41
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.142243 -> initscore=-1.796785
[LightGBM] [Info] Start training from score -1.796785
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 397, number of negative: 2394
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000518 seconds.
You can set `force_row_wise=true` t

[I 2024-08-18 16:21:12,408] Trial 0 finished with value: 0.8293935495963878 and parameters: {'n_estimators': 4600, 'learning_rate': 0.056908769933251474, 'colsample_bytree': 0.5840907599010865, 'subsample': 0.8437817320143863}. Best is trial 0 with value: 0.8293935495963878.


[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 397, number of negative: 2394
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000546 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 683
[LightGBM] [Info] Number of data points in the train set: 2791, number of used features: 41
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.142243 -> initscore=-1.796785
[LightGBM] [Info] Start training from score -1.796785
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 397, number of negative: 2394
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000506 seconds.
You can set `force_row_wise=true` t

[I 2024-08-18 16:21:15,906] Trial 1 finished with value: 0.8251752858369684 and parameters: {'n_estimators': 2574, 'learning_rate': 0.004077761250969111, 'colsample_bytree': 0.7152525240096559, 'subsample': 0.7590287547349139}. Best is trial 0 with value: 0.8293935495963878.


...

[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 397, number of negative: 2394
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000530 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 683
[LightGBM] [Info] Number of data points in the train set: 2791, number of used features: 41
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.142243 -> initscore=-1.796785
[LightGBM] [Info] Start training from score -1.796785
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 397, number of negative: 2394
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000472 seconds.
You can set `force_row_wise=true` t

[I 2024-08-18 16:22:44,262] Trial 49 finished with value: 0.8262368577483986 and parameters: {'n_estimators': 375, 'learning_rate': 0.001460585657990403, 'colsample_bytree': 0.6112137818471376, 'subsample': 0.6477286543850995}. Best is trial 14 with value: 0.8321789736156433.
[I 2024-08-18 16:22:44,268] A new study created in memory with name: no-name-3fd307e3-b940-4ad1-9de9-8e3bbf94eecd


Best LightGBM parameters: {'n_estimators': 851, 'learning_rate': 0.02289938761734719, 'colsample_bytree': 0.5046482989824799, 'subsample': 0.519032610565576}


[I 2024-08-18 16:23:04,431] Trial 0 finished with value: 0.8358587243730599 and parameters: {'iterations': 1168, 'learning_rate': 0.009643296892387335, 'colsample_bylevel': 0.9776542079533395}. Best is trial 0 with value: 0.8358587243730599.
[I 2024-08-18 16:23:14,333] Trial 1 finished with value: 0.8219887758157338 and parameters: {'iterations': 628, 'learning_rate': 0.00010885687635135775, 'colsample_bylevel': 0.5104925513663798}. Best is trial 0 with value: 0.8358587243730599.
[I 2024-08-18 16:24:13,608] Trial 2 finished with value: 0.836689377005774 and parameters: {'iterations': 3583, 'learning_rate': 0.004637085218419185, 'colsample_bylevel': 0.7070364208604409}. Best is trial 2 with value: 0.836689377005774.
[I 2024-08-18 16:24:38,480] Trial 3 finished with value: 0.8372961413510879 and parameters: {'iterations': 1745, 'learning_rate': 0.002918396056771357, 'colsample_bylevel': 0.5533842737399985}. Best is trial 3 with value: 0.8372961413510879.
[I 2024-08-18 16:25:34,650] Trial

Best CatBoost parameters: {'iterations': 2071, 'learning_rate': 0.010918521113089146, 'colsample_bylevel': 0.5022508048518279}
Final LightGBM Parameters: {'n_estimators': 851, 'learning_rate': 0.02289938761734719, 'colsample_bytree': 0.5046482989824799, 'subsample': 0.519032610565576}
Final CatBoost Parameters: {'iterations': 2071, 'learning_rate': 0.010918521113089146, 'colsample_bylevel': 0.5022508048518279}
Training LightGBM...
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Number of positive: 397, number of negative: 2394
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000396 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 683
[LightGBM] [Info] Number of data points in the train set: 2791, number of used features: 41
[LightGBM] [Warning] Found whitespace in feature_names, replace wi

In [19]:
meta_features

array([[0.79405532, 0.7972722 ],
       [0.13327945, 0.12743126],
       [0.22523849, 0.16892445],
       ...,
       [0.04776546, 0.03370423],
       [0.11371424, 0.23595106],
       [0.06842982, 0.08301541]])

In [20]:
meta_features

array([[0.79405532, 0.7972722 ],
       [0.13327945, 0.12743126],
       [0.22523849, 0.16892445],
       ...,
       [0.04776546, 0.03370423],
       [0.11371424, 0.23595106],
       [0.06842982, 0.08301541]])

In [21]:
params_logistic = {
    "penalty": 'l2',             # L2正則化
    "C": 1.0,                    # 正則化の強さ
    "solver": 'lbfgs',           # 最適化アルゴリズム
    "max_iter": 100,             # 最大反復回数
    #"class_weight":'balanced',    # クラスの重みづけ
    "random_state": 1506            # 乱数シード
}

In [22]:
# メタモデルの定義と学習
from sklearn.linear_model import LogisticRegression
meta_model = LogisticRegression(**params_logistic)
oof_predictions_meta = train_meta_model(meta_features, train_y, meta_model)


Meta Model Fold 1, AUC: 0.8181
Meta Model Fold 2, AUC: 0.8482
Meta Model Fold 3, AUC: 0.8387
Meta Model Fold 4, AUC: 0.8199
Meta Model Fold 5, AUC: 0.8622
Meta Model OOF AUC: 0.8364


Meta Model Fold 1, AUC: 0.8172
Meta Model Fold 2, AUC: 0.8446
Meta Model Fold 3, AUC: 0.8359
Meta Model Fold 4, AUC: 0.8211
Meta Model Fold 5, AUC: 0.8641
Meta Model OOF AUC: 0.8356

In [23]:
final_auc = roc_auc_score(train_y, oof_predictions_meta)
print(f"Final Stacking Model OOF AUC: {final_auc:.8f}")

Final Stacking Model OOF AUC: 0.83640815


In [27]:
base_models.items()

dict_items([('LightGBM', LGBMClassifier(colsample_bytree=0.5046482989824799,
               learning_rate=0.02289938761734719, n_estimators=851,
               subsample=0.519032610565576)), ('CatBoost', <catboost.core.CatBoostClassifier object at 0x7d57084f4a90>)])

In [31]:
test_predictions = stacking_predict(test_x, base_models, meta_model)

In [32]:
test_predictions

array([0.07317038, 0.09364267, 0.14600888, ..., 0.58701951, 0.09627975,
       0.05162425])

In [33]:
count = 3

In [34]:
from datetime import datetime
import pytz
# カウント変数をインクリメント
count += 1
# 日本時間を取得
japan_tz = pytz.timezone('Asia/Tokyo')
now = datetime.now(japan_tz)
timestamp = now.strftime("%Y%m%d_%H%M%S")

file_name = f"/content/drive/MyDrive/Signate/2024summer/lightcatxgbop_en_{timestamp}_{count:03d}.csv"
ss[1] = test_predictions
ss.to_csv(file_name, header=False, index=False)